In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import geopandas as gpd
import pandas as pd
from pathlib import Path

/home/abbymoreno/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [3]:
# custom module
import sys

sys.path.append("../../../")  # include parent directory
from src.nightlights import *

# Extract Nightlights

### Set-up inputs and parameters

In [4]:
ADMIN_FPATH = Path("../../../data/01-admin-bounds/")
OUTPUT_DIR = Path("../../../data/04-output/nightlights/")
NTL_DIR = Path("../../../data/02-raw/nightlights/")

### Load AOI

In [5]:
# Load the administrative boundaries for the 12 cities
aoi = gpd.read_file(ADMIN_FPATH / "target_admin_bounds.shp")
aoi.head(2)

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053..."
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039..."


In [6]:
# Log-in using EOG credentials
username = os.environ.get("EOG_USER", None)
username = username if username is not None else input("Username?")
password = os.environ.get("EOG_PASSWORD", None)
password = password if password is not None else getpass.getpass("Password?")

# set save_token to True so that access token gets stored in ~/.eog_creds/eog_access_token
access_token = get_eog_access_token(username, password, save_token=True)

2024-02-02 13:25:04.116 | INFO     | src.nightlights:get_eog_access_token:42 - Saving access_token to /home/abbymoreno/.eog_creds/eog_access_token.txt
2024-02-02 13:25:04.119 | INFO     | src.nightlights:get_eog_access_token:50 - Adding access token to environment var EOG_ACCESS_TOKEN


In [7]:
def download_nightlights(aoi, year=2021):
    nightlights_aoi = generate_nightlights_feature(aoi, year)
    nightlights_df = nightlights_aoi.drop(
        columns=[
            "ADM1_EN",
            "ADM1_PCODE",
            "ADM2_EN",
            "ADM2_PCODE",
            "ADM3_EN",
            "ADM3_PCODE",
            "ADM4_EN",
            "geometry",
        ]
    )
    nightlights_df = pd.DataFrame(nightlights_df)
    nightlights_df.insert(1, "date", f"{year}-01-01")
    nightlights_df.insert(2, "freq", "Y")
    nightlights_df.to_csv(OUTPUT_DIR / f"nightlights_{year}.csv", index=False)

In [ ]:
download_nightlights(aoi, 2018)